In [1]:
import pandas as pd
import geopandas as gpd
import json
from tqdm.auto import tqdm

In [2]:
total_impacts_file = "/home/colin/Downloads/NIH_impact_county (2).csv"
total_impacts = pd.read_csv(total_impacts_file, encoding='latin-1')

In [9]:
total_impacts

,FIPS_live,state,county,pop_2024,grant_funds,grant_funds_econ,econ_loss,IDC_loss,jobs_loss,grant_funds_econ_25k,econ_loss_25k,IDC_loss_25k,grant_funds_econ_pc,econ_loss_pc,IDC_loss_pc,grant_funds_econ_log,econ_loss_log,IDC_loss_log,fiscal_year,FIPS
0,1001,Alabama,Autauga County,61464,4.128291e+05,1.056843e+06,1.491193e+05,5.824974e+04,0.644069,4.298624e+05,6.065312e+04,23692.625370,17.194498,2.426125,0.947705,13.870797,11.912509,10.972512,2024,1001
1,1003,Alabama,Baldwin County,261608,1.020089e+06,2.611427e+06,4.923642e+05,1.923297e+05,2.126594,2.495554e+05,4.705171e+04,18379.574436,9.982214,1.882068,0.735183,14.775408,13.106976,12.166972,2024,1003
2,1005,Alabama,Barbour County,24358,7.215050e+03,1.847053e+04,3.076767e+03,1.201862e+03,0.013289,1.895735e+04,3.157861e+03,1233.539422,0.758294,0.126314,0.049342,9.823986,8.031960,7.092459,2024,1005
3,1007,Alabama,Bibb County,22258,2.021568e+06,5.175213e+06,6.950520e+05,2.715047e+05,3.002033,5.812756e+06,7.806766e+05,304951.812102,232.510240,31.227066,12.198072,15.459391,13.451743,12.511738,2024,1007
4,1009,Alabama,Blount County,60163,9.256771e+06,2.369733e+07,3.189213e+06,1.245786e+06,13.774687,9.847138e+06,1.325239e+06,517671.327924,393.885506,53.009544,20.706853,16.980873,14.975285,14.035278,2024,1009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,56037,Wyoming,Sweetwater County,41273,9.545280e+03,2.443592e+04,3.830357e+03,1.496233e+03,0.016544,1.480139e+04,2.320135e+03,906.302576,0.592056,0.092805,0.036252,10.103850,8.250974,7.311374,2024,56037
3140,56039,Wyoming,Teton County,23272,1.345964e+05,3.445669e+05,5.532538e+04,2.161148e+04,0.238959,3.701518e+05,5.943342e+04,23216.178904,14.806071,2.377337,0.928647,12.750046,10.921005,9.981026,2024,56039
3141,56041,Wyoming,Uinta County,20621,8.542935e+04,2.186991e+05,3.610118e+04,1.410202e+04,0.155926,2.651413e+05,4.376750e+04,17096.678103,10.605651,1.750700,0.683867,12.295457,10.494109,9.554145,2024,56041
3142,56043,Wyoming,Washakie County,7662,1.085214e+04,2.778148e+04,4.890759e+03,1.910453e+03,0.021124,9.064695e+04,1.595784e+04,6233.530889,3.625878,0.638314,0.249341,10.232161,8.495307,7.555619,2024,56043


In [3]:
counties = gpd.read_file('/home/colin/code/scimap-data/inputs/merged_data_counties_CLIP_Compress.geojson') # Example, replace with your shapefile path
counties = counties.to_crs(epsg=4326) # Project to WGS84 (lat/lon)

In [4]:
total_impacts["FIPS"] = total_impacts["FIPS_live"]

In [5]:
merged_df_county_total = pd.merge(total_impacts, counties[['FIPS', 'geometry']], on='FIPS', how='left')


In [6]:
features = []

for index, row in tqdm(merged_df_county_total.iterrows(), desc="Processing into GeoJSON (IDC COUNTY)", total=len(merged_df_county_total)):
    # Convert the Shapely geometry to GeoJSON format
    geometry = json.loads(gpd.GeoSeries([row['geometry']]).to_json())
    geometry = geometry['features'][0]['geometry']

    # Create a feature dictionary
    feature = {
        "type": "Feature",
        "geometry": geometry,
        "properties": row.drop('geometry').to_dict() # Include all columns except 'geometry'
    }
    features.append(feature)

# Create the GeoJSON feature collection
geojson = {
    "type": "FeatureCollection",
    "features": features
}

# Save the GeoJSON to a file
with open("/home/colin/code/scimap-data/outputs/merged_data_total_impacts_counties.geojson", "w") as f:
    json.dump(geojson, f)

Processing into GeoJSON (IDC COUNTY):   0%|          | 0/3144 [00:00<?, ?it/s]

In [7]:
print("Processing Counties (IDC)")
!tippecanoe -z7 -e tiles_counties_total_exp --no-tile-compression outputs/merged_data_total_impacts_counties.geojson

Processing Counties (IDC)
For layer 0, using name "merged_data_total_impacts_counties"
3144 features, 6711113 bytes of geometry and attributes, 844897 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
tile 0/0/0 size is 837539 (probably really 837539) with detail 12, >500000    
tile 0/0/0 size is 790202 (probably really 790202) with detail 11, >500000    
tile 1/0/0 size is 861973 (probably really 861973) with detail 12, >500000    
tile 1/0/0 size is 837857 (probably really 837857) with detail 11, >500000    
tile 1/0/0 size is 789784 (probably really 789784) with detail 10, >500000    
tile 2/0/1 size is 517571 (probably really 517571) with detail 12, >500000    
  99.8%  7/33/50  


In [8]:
!rclone copy tiles_counties_total_exp/ r2:scimap-data/tiles_counties_total_exp/ --transfers 32
